In [1]:
%matplotlib inline

# join one or more path components intelligently
from os.path import join

#torch import
import torch
from torch.utils.data import random_split
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

# Interface com o sistema operacional
import os

# Manipulação de dataframes
import pandas as pd

# Manipulação de dados tabulares
import numpy as np

# Normalização das características
from sklearn.preprocessing import StandardScaler

# visualização de dados baseada no matplotlib
import seaborn as sns

# Esboço de gráficos
from matplotlib import pyplot as plt

# Classificação de clientes

## Carga e inspeção dos dados

In [2]:
# Definição dos nomes das variáveis (conforme a tabela contida no enunciado)
colnames = ['ESCT', 'NDEP', 'RENDA', 'TIPOR', 'VBEM', 'NPARC',
            'VPARC', 'TEL', 'IDADE', 'RESMS', 'ENTRADA', 'CLASSE']

In [3]:
# Leitura dos dados de treino
arquivo = '/content/sample_data/credtrain.txt'
data_train = pd.read_csv(arquivo, sep='\t', header=None, names = colnames)

# Leitura dos dados de teste
arquivo = '/content/sample_data/credtest.txt'
data_test = pd.read_csv(arquivo, sep='\t', header=None, names = colnames)

**Inspeção dos dados**

In [4]:
# Inspeção da dimensão do dataset
print(data_train.shape, data_test.shape)

(1500, 12) (577, 12)


In [5]:
# Inspeção das primeiras linhas do conjunto de treinamento
data_train.head()

,ESCT,NDEP,RENDA,TIPOR,VBEM,NPARC,VPARC,TEL,IDADE,RESMS,ENTRADA,CLASSE
0,1,0,360,0,313,9,52,0,25,48,0,1
1,0,0,350,1,468,10,65,0,33,6,0,1
2,0,0,1100,0,829,9,125,0,56,48,0,1
3,0,0,3000,0,552,12,76,1,31,60,0,1
4,1,0,1000,0,809,12,111,0,24,7,0,1


In [6]:
# Inspeção das primeiras linhas do conjunto de teste
data_test.head()

,ESCT,NDEP,RENDA,TIPOR,VBEM,NPARC,VPARC,TEL,IDADE,RESMS,ENTRADA,CLASSE
0,0,2,500,1,618,10,85,0,36,6,0,0
1,1,0,813,0,552,4,119,0,43,48,119,1
2,3,0,350,0,488,12,66,0,43,0,0,1
3,1,0,1530,0,381,1,398,0,28,48,0,1
4,0,0,688,1,396,10,60,0,49,72,0,1


## Pré-processamento dos dados

### Transformação de variáveis não-numéricas

É importante observar que a variável ESCT (Estado Civil) é do tipo categórica, podendo assumir 4 valores diferentes (cada valor corresponde a um estado civil). Assim, diferentemente de NDEP (onde cada valor corresponde a uma quantidade de dependentes), na variável ESCT cada valor corresponde a uma categoria. Contudo, este fato pode trazer inconsistências na criação e treinamento de modelos.

Para mitigar este problema, uma alternativa é tranformar a variável ESCT em uma variável *dummy* (variável binária). Neste sentido, cada categoria da variável ESCT corresponderá a uma variável. Visto que há 4 possíveis categorias para a variável ESCT, obteremos 4 variáveis ESCT binárias.

Uma variável *dummy* é uma variável binária utilizadas para representar categorias. Neste sentido, em um caso de uma variável com 3 ou mais categorias, recomenda-se a criação de $n-1$ dummies. Diante disso, a variável ESCT será transformada em 4 "variantes dummy", onde o valor 1 corresponderá à ocorrência de determinada categoria e o valor 0 corresponderá à não ocorrência.

In [7]:
# Aplicação no conjunto de treinamento
data_train_new = pd.get_dummies(data = data_train, 
                                prefix='ESCT', 
                                columns=['ESCT'], 
                                drop_first=True)

"""
pd.get_dummies: Convert categorical variable into dummy/indicator variables.
"""

# Inspeção das primeiras linhas
data_train_new.head()

,NDEP,RENDA,TIPOR,VBEM,NPARC,VPARC,TEL,IDADE,RESMS,ENTRADA,CLASSE,ESCT_1,ESCT_2,ESCT_3
0,0,360,0,313,9,52,0,25,48,0,1,1,0,0
1,0,350,1,468,10,65,0,33,6,0,1,0,0,0
2,0,1100,0,829,9,125,0,56,48,0,1,0,0,0
3,0,3000,0,552,12,76,1,31,60,0,1,0,0,0
4,0,1000,0,809,12,111,0,24,7,0,1,1,0,0


In [8]:
# Aplicação da transformação sobre o conjunto de teste
data_test_new = pd.get_dummies(data = data_test, prefix='ESCT', columns=['ESCT'], drop_first=True)

"""
pd.get_dummies: Convert categorical variable into dummy/indicator variables
"""

# Inspeção das primeiras linhas
data_test_new.head()

,NDEP,RENDA,TIPOR,VBEM,NPARC,VPARC,TEL,IDADE,RESMS,ENTRADA,CLASSE,ESCT_1,ESCT_2,ESCT_3
0,2,500,1,618,10,85,0,36,6,0,0,0,0,0
1,0,813,0,552,4,119,0,43,48,119,1,1,0,0
2,0,350,0,488,12,66,0,43,0,0,1,0,0,1
3,0,1530,0,381,1,398,0,28,48,0,1,1,0,0
4,0,688,1,396,10,60,0,49,72,0,1,0,0,0


**Separação do conjunto de dados em rótulo ($\mathrm{y}$) e features ($\mathrm{x}$)**

O rótulo ($\mathrm{y}$) corresponde ao vetor contendo a variável alvo (CLASSE), enquanto que features ($\mathrm{x}$) corresponde à matriz de dados.

In [9]:
# Transformação da variável alvo do conjunto de treinamento em vetor
y_train = np.array(data_train_new['CLASSE'])

# Inspeção das primeira linhas
y_train[:5]

array([1, 1, 1, 1, 1])

In [10]:
# Transformação da variável alvo do conjunto de teste em vetor
y_test = np.array(data_test_new['CLASSE'])

# Inspeção das primeiras linhas
y_test[:5]

array([0, 1, 1, 1, 1])

In [11]:
# Transformação do conjunto de treinamento remanescente em matriz de dados
features_name_train = list(data_train_new.columns)               # nomes das colunas
features_name_train.remove('CLASSE')                             # remove variável "CLASSE"
X_train = np.array(data_train_new.loc[:, features_name_train])   # Transformação em matriz de dados

# Inspeção da matriz resultante
X_train

array([[   0,  360,    0, ...,    1,    0,    0],
       [   0,  350,    1, ...,    0,    0,    0],
       [   0, 1100,    0, ...,    0,    0,    0],
       ...,
       [   0,  570,    0, ...,    0,    0,    0],
       [   0,  360,    0, ...,    0,    0,    0],
       [   4,  501,    1, ...,    0,    0,    0]])

In [12]:
# Transformação do conjunto de teste remanescente em matriz de dados
features_name_test = list(data_test_new.columns)               # Recuperação dos nomes das colunas
features_name_test.remove('CLASSE')                            # Remoção da variável "CLASSE"
X_test = np.array(data_test_new.loc[:, features_name_test])   # Transformação em matriz

# Inspeção da matriz resultante
X_test

array([[   2,  500,    1, ...,    0,    0,    0],
       [   0,  813,    0, ...,    1,    0,    0],
       [   0,  350,    0, ...,    0,    0,    1],
       ...,
       [   3, 1200,    0, ...,    0,    0,    0],
       [   0,  600,    0, ...,    1,    0,    0],
       [   0,  800,    1, ...,    0,    0,    0]])

### Normalização das features

Antes de iniciar o treinamento, é também necessário realizar a *normalização* das características a fim de evitar problemas decorrentes à discrepância nas ordens de grandeza das features.

In [13]:
# Criação do objeto para a padronização das features
scaler = StandardScaler()

# Ajustamento do StandardScaler ao conjunto de dados de treino e padronização dos dados de treino
X_train_norm = scaler.fit_transform(X_train)

# Transformação dos dados de teste com os parâmetros ajustados a partir dos dados de treino
X_test_norm = scaler.transform(X_test)

In [14]:
# Dimensões dos datasets
print(X_train_norm.shape, X_test_norm.shape, y_train.shape, y_test.shape)

(1500, 13) (577, 13) (1500,) (577,)


## Treinamento do modelo

No trecho de código a seguir, foi realizada uma transformação dos dados em vetores para tensores.
Esses dados ficaram armazenados nas variáveis x_train, y_train, x_test, y_test.

Essa transformação foi feita porque posteriormente iremos utiliza-las como parâmetro para nosso dataloader.

In [15]:
y_train.shape

(1500,)

In [16]:
# y_train.reshape(-1, 1)
# y_test.reshape(-1, 1)

# print(y_train.shape, y_test.shape)

In [17]:
x_train = torch.from_numpy(X_train_norm)
x_test = torch.from_numpy(X_test_norm)

y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

xtr, ytr, xte, yte = x_train.float(), y_train.float(), x_test.float(), y_test.float()

print(xtr.shape)
print(ytr.shape)

print(xte.shape)
print(yte.shape)


torch.Size([1500, 13])
torch.Size([1500])
torch.Size([577, 13])
torch.Size([577])


In [18]:
# Criação do objeto para a padronização das features
scaler_for_y = StandardScaler()

# Ajuste do StandardScaler ao conjunto de dados de treino e padronização dos dados de treino
#y_train_norm = scaler_for_y.fit_transform(y_train)

# Transformação dos dados de teste com os parâmetros ajustados a partir dos dados de treino
#y_test_norm = scaler_for_y.transform(y_test)

A seguir, importamos a classe `TensorDataset`, responsável por criar as instancias do tipo tensor para os vetores x_train, y_train e x_test, y_test. Os `TensorDataset` ficaram armezenados nas variáveis train_ds e val_ds, que são respectivamente, datasets de treinamento e avaliação.

In [19]:
from torch.utils.data.dataset import TensorDataset

train_ds = TensorDataset(xtr, ytr)
val_ds = TensorDataset(xte, yte)

Abaixo temos o código necessário para verificar a taxa de acurácia da nossa rede. Esse método será necessário para verificar quão bom foi o processo de aprendizado da rede. Ele será invocado a partir do método `fit`, que será implementado mais a frente.

In [20]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

Abaixo, temos um código que realiza a criação dos batchs de dados. Um batch de dados é basicamente um conjunto menor de dados.
Passamos como parâmetro um dataset inteiro, e criamos inúmeros datasets menores de tamanho N, onde N corresponde ao valor da váriavel batch_size do código abaixo. Com o objeto DataLoader conseguimos criar batchs de tamanho N a partir de um conjunto de dados de entrada. O parâmetro `shuffle = True` signfica que os dados de treinamento serão embaralhados para garantir que os lotes gerados em cada época sejam diferentes.

In [22]:
batch_size = 40
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

Abaixo temos a implementação da classe LogisticModel, que estende a classe Module do pacote nn. Essa classe carrega a lógica de treinamento e avaliação da rede, e apresenta uma lista de métodos:

- `training_step`: realiza as predições e calcula as perdas durante a fase de treinamento.

- `validation_step`: realiza as predições, calcula as perdas e verifica a taxa de acurácia durante a fase de avaliação.

- `validation_epoch_end`: retornar as perdas e as acurácias.

- `epoch_end`: imprime as perdas e acurácias obtidas ao fim de uma época de treinamento.

In [23]:
input_size = 13
num_classes = 2

In [24]:
class LogisticModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, num_classes).float()
        
    def forward(self, xb):
        
        out = self.linear(xb)
        return out
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  

        # Veja https://discuss.pytorch.org/t/runtimeerror-expected-object-of-scalar-type-long-but-got-scalar-type-float-when-using-crossentropyloss/30542
        labels = labels.long()

        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    

        # Veja https://discuss.pytorch.org/t/runtimeerror-expected-object-of-scalar-type-long-but-got-scalar-type-float-when-using-crossentropyloss/30542
        labels = labels.long()

        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss, 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))
    


In [25]:
model = LogisticModel()
print(model.parameters)

<bound method Module.parameters of LogisticModel(
  (linear): Linear(in_features=13, out_features=2, bias=True)
)>


In [26]:
def evaluate(model, val_loader):
  outputs = [model.validation_step(batch) for batch in val_loader]
  return model.validation_epoch_end(outputs)

A função `fit` registra o custo medido sobre o conjunto de validação e a métrica de cada época. Ele retorna um histórico do treinamento, útil para depuração e visualização. Nesse método passamos configurações iniciais como taxa de aprendizado, tamanho de lote e etc. Essas configurações são denominadas hiperparâmetros. Esses hiperparâmetros possuem uma relevância muito grande no contexto de aprendizado de máquina. Dentro deste método fit, há ainda o método `evaluate`, responsável por avaliar a rede neural durante a fase de validação.

In [27]:
def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    optimizer = opt_func(model.parameters(), lr)
    history = [] # for recording epoch-wise results
    
    for epoch in range(epochs):
        
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result)
        history.append(result)

    return history
  

In [28]:
result0 = evaluate(model, val_loader)
result0

{'val_acc': 0.3329411745071411, 'val_loss': 0.8614569902420044}

In [29]:
history1 = fit(300, 0.001, model, train_loader, val_loader)

Epoch [0], val_loss: 0.8432, val_acc: 0.3363
Epoch [1], val_loss: 0.8257, val_acc: 0.3346
Epoch [2], val_loss: 0.8091, val_acc: 0.3379
Epoch [3], val_loss: 0.7932, val_acc: 0.3613
Epoch [4], val_loss: 0.7778, val_acc: 0.3863
Epoch [5], val_loss: 0.7632, val_acc: 0.4029
Epoch [6], val_loss: 0.7493, val_acc: 0.4213
Epoch [7], val_loss: 0.7360, val_acc: 0.4452
Epoch [8], val_loss: 0.7233, val_acc: 0.4752
Epoch [9], val_loss: 0.7111, val_acc: 0.4885
Epoch [10], val_loss: 0.6994, val_acc: 0.5069
Epoch [11], val_loss: 0.6883, val_acc: 0.5285
Epoch [12], val_loss: 0.6777, val_acc: 0.5652
Epoch [13], val_loss: 0.6676, val_acc: 0.6141
Epoch [14], val_loss: 0.6578, val_acc: 0.6453
Epoch [15], val_loss: 0.6485, val_acc: 0.6686
Epoch [16], val_loss: 0.6396, val_acc: 0.6903
Epoch [17], val_loss: 0.6311, val_acc: 0.7103
Epoch [18], val_loss: 0.6230, val_acc: 0.7253
Epoch [19], val_loss: 0.6152, val_acc: 0.7375
Epoch [20], val_loss: 0.6078, val_acc: 0.7604
Epoch [21], val_loss: 0.6006, val_acc: 0.767

In [30]:
history2 = fit(20, 0.001, model, train_loader, val_loader)

Epoch [0], val_loss: 0.5816, val_acc: 0.7910
Epoch [1], val_loss: 0.5753, val_acc: 0.7960
Epoch [2], val_loss: 0.5691, val_acc: 0.8076
Epoch [3], val_loss: 0.5632, val_acc: 0.8093
Epoch [4], val_loss: 0.5575, val_acc: 0.8126
Epoch [5], val_loss: 0.5520, val_acc: 0.8160
Epoch [6], val_loss: 0.5467, val_acc: 0.8210
Epoch [7], val_loss: 0.5416, val_acc: 0.8226
Epoch [8], val_loss: 0.5366, val_acc: 0.8260
Epoch [9], val_loss: 0.5318, val_acc: 0.8293
Epoch [10], val_loss: 0.5272, val_acc: 0.8310
Epoch [11], val_loss: 0.5228, val_acc: 0.8343
Epoch [12], val_loss: 0.5185, val_acc: 0.8343
Epoch [13], val_loss: 0.5144, val_acc: 0.8360
Epoch [14], val_loss: 0.5104, val_acc: 0.8343
Epoch [15], val_loss: 0.5065, val_acc: 0.8376
Epoch [16], val_loss: 0.5028, val_acc: 0.8466
Epoch [17], val_loss: 0.4992, val_acc: 0.8449
Epoch [18], val_loss: 0.4957, val_acc: 0.8466
Epoch [19], val_loss: 0.4923, val_acc: 0.8482


In [31]:
history3 = fit(20, 0.001, model, train_loader, val_loader)

Epoch [0], val_loss: 0.4891, val_acc: 0.8499
Epoch [1], val_loss: 0.4859, val_acc: 0.8482
Epoch [2], val_loss: 0.4828, val_acc: 0.8532
Epoch [3], val_loss: 0.4798, val_acc: 0.8549
Epoch [4], val_loss: 0.4770, val_acc: 0.8588
Epoch [5], val_loss: 0.4742, val_acc: 0.8588
Epoch [6], val_loss: 0.4715, val_acc: 0.8588
Epoch [7], val_loss: 0.4688, val_acc: 0.8605
Epoch [8], val_loss: 0.4663, val_acc: 0.8622
Epoch [9], val_loss: 0.4638, val_acc: 0.8622
Epoch [10], val_loss: 0.4615, val_acc: 0.8605
Epoch [11], val_loss: 0.4591, val_acc: 0.8638
Epoch [12], val_loss: 0.4568, val_acc: 0.8638
Epoch [13], val_loss: 0.4546, val_acc: 0.8638
Epoch [14], val_loss: 0.4525, val_acc: 0.8672
Epoch [15], val_loss: 0.4504, val_acc: 0.8672
Epoch [16], val_loss: 0.4484, val_acc: 0.8672
Epoch [17], val_loss: 0.4464, val_acc: 0.8672
Epoch [18], val_loss: 0.4445, val_acc: 0.8655
Epoch [19], val_loss: 0.4426, val_acc: 0.8672


## Validação do modelo

In [30]:
test_dataset = TensorDataset(xte, yte)
test_loader = DataLoader(test_dataset, batch_size=40)
result = evaluate(model, test_loader)
result


{'val_acc': 0.883823573589325, 'val_loss': 0.34420245885849}